# CMSE 491 Final Project

###  Kyle Taft
#### December 4, 2023

<img src="https://github.com/KyleTaft/Fall23/blob/main/CMSE491/gamma-radiation-clipart.png?raw=true" alt="Gamma" width="700"/>

[Source](https://www.pinclipart.com/maxpin/bioxxT/)

## INSTRUCTIONS FOR RUNNING THE NOTEBOOK

* This notebook is used to train and evaluate models that extract nuclear decay energy peaks from 2D total absorption spectra (TAS) data. In order to produce enough data for machine learning, the data is simulated using the open source physics toolkit [Geant4](https://geant4.web.cern.ch/). Unfortunately, given the nature of the data being tens of thousands of matrices of size 512x512, the data is far too large to be attached to this notebook or uploaded to D2L in full. However, a small sample set of 15 matrices (5 single cascades, 5 double cascades, 5 two independent single cascades) and their corresponding labels are provided in the file `sample_data.npz` in the same folder as this notebook. The data can be loaded using the following code:

```python
import numpy as np
data = np.load("sample_data.npz")
```

* The data is stored in a dictionary, with the keys being the names of the matrices. The matrices are stored as numpy arrays, and can be accessed using the following code:

```python
matrix = data["matrix_name"]
```




### **tensorflow_addons**



# **Utilizing Machine Learning to Perform Total Absorption Spectroscopy**

## Background and Motivation

The analysis and interpretation of experimental data is an integral part of the scientific process, enabling us to uncover the underlying story held by the experiment. The method of processing this data has always gone hand in hand in utilization and improvement of cutting-edge technology. This is no exception in the modern era which is dominated by exponentially improving computer hardware and machine learning algorithms.

In experimental physics, analyzing the spectrum of emitted light, a potent and accessible tool known as spectroscopy, serves to unravel the underlying physics governing an experiment, leading to valuable insights into the unknown phenomena at play. In nuclear physics research, spectroscopy is indispensable for analyzing the decay of exotic isotopes. This is because when an isotope decays into its daughter, it often leaves the new daughter isotope with an excess of energy. As a fundamental principle in physics, systems tend towards their lowest energy state and because of quantum mechanics, nuclides do not decay continuously, but instead emit discrete energy packets to achieve this favorable state of energy. This method of shedding energy can be through many means, such as ejecting a particle like a proton, neutron, or alpha particle, converting a proton or neutron into the other through beta decay, or through isomeric transition – the nucleus simply releasing energy through light (gamma rays). While whole sections of nuclear physics are dedicated to creating a deep understanding of each one of these possible methods, the topic of this project specifically focuses on these gamma rays.

When a nucleus emits a gamma ray it does so in accordance with selection rules and its own complex structure. This is all to say that this matter is not very simple nor predictable. Except for a subset of simplistic cases, often the isotope has multiple discrete excitation levels that each might have different paths of releasing gamma rays and reaching the ground state. To create a clearer explanation, I explain a fake example below.



### EXAMPLE DECAY SCHEME


<img src="https://github.com/KyleTaft/Fall23/blob/main/CMSE491/Example%20Decay%20Scheme.png?raw=true" alt="Decay" width="400"/>


### EXPLAIN TAS WITH GEANT SIMS OF DECAY SCHEME


### EXPLAIN DATA COLLECTION WITH SUN


### EXPLAIN WHY THIS IS IMPORTANT (NUCLEAR  STRUCTURE, NUCLEAR ASTROPHYSICS, MEDICAL IMAGING, ETC.)

### WHY MACHINE LEARNING IS IMPORTANT (AUTOMATION, SPEED, ETC.)

## Goal
The goal of my project is to develop a model that accurately predicts the discrete energy levels, gamma-ray transitions, and level intensities of a given experimental or simulated nuclear decay spectrum.

## Data Simulation and Preprocessing

### Explain Geant4

### Show sample input file

### Show the attached sample data

### How I make my labels

### HONORS Superposition stuff

In [ ]:
# Data Cleaning Example
import numpy as np
def transform_label(label, single = True):
    """
    label: a single label from the dataset
    single: boolean, True if single, False if double
    :return: the coordinates of energies in the label
    """
    cord = np.unravel_index(np.argmax(label, axis=None), label.shape)[0:2] # find the max
    if single:
        cord = np.array([20*cord[1], 20*(511-cord[0])]) # convert to energy
        return np.pad(cord, (0, 2), 'constant', constant_values=(-1,-1)) # pad with -1 to make it 2x2 matrix using unphysical values
    else:
        # find the second max
        label[cord[0], cord[1]] = 0
        cord2 = np.unravel_index(np.argmax(label, axis=None), label.shape)[0:2]
        return np.array([20*cord[1], 20*(511-cord[0]), 20*cord2[1], 20*(511-cord2[0])])

#Example labels
single_label = np.load("/mnt/home/taftkyle/indiv_data/singles/single_test_label.npy")[0]
double_label = np.load("/mnt/home/taftkyle/indiv_data/doubles/double_test_label.npy")[1]

print("Single Label: \n", transform_label(single_label, ).reshape(2,2))
print("Double Label: \n", transform_label(double_label, single = False).reshape(2,2))

In [ ]:
# Visualize the data (example spectra)
import matplotlib.pyplot as plt

# Function to convert the matrix to energy values and counts:
def matrix_to_coords(array):
    xys = []
    values = []
    for i in range(array.shape[0]):
        for j in range(array.shape[1]):
            xys.append((20*j,20*(511-i)))
            values.append(array[i,j])
    return np.asarray(xys), np.asarray(values)

# Load the first data point of test data:
single = np.load("/mnt/home/taftkyle/indiv_data/singles/raw_single_test.npy")[0]
double = np.load("/mnt/home/taftkyle/indiv_data/doubles/raw_double_test.npy")[1] # I loaded the second double because the first one is not as nice to visualize
single_label= np.load("/mnt/home/taftkyle/indiv_data/singles/single_test_label.npy")[0]
double_label = np.load("/mnt/home/taftkyle/indiv_data/doubles/double_test_label.npy")[1]

In [ ]:
# Plot the data:
for example in [[single, single_label], [double, double_label]]:
    # Separate the data and label:
    data = example[0]
    label = example[1]

    #replace all values less than 1 with nan in order to appear white on graph
    data = data.astype(float)
    label = label.astype(float)
    data[data < 1.] = np.nan
    label[label < 1.] = np.nan

    # Subplot 1: Plot the raw data
    plt.figure(figsize=(20,10))
    plt.subplot(1,2,1)
    plt.title("Raw Data")
    plt.xlabel("Individual Energy (keV)")
    plt.ylabel("Summed Energy (keV)")
    points, vals = matrix_to_coords(data)
    plt.scatter(points[:,0],points[:,1], marker='s', c=vals, cmap='viridis', label="High Value Pixels", s=1)
    cbar = plt.colorbar()
    plt.xlim(0,10000)
    plt.ylim(0,10000)

    # Subplot 2: Plot the labels
    plt.subplot(1,2,2)
    plt.title("Label")
    plt.xlabel("Individual Energy (keV)")
    plt.ylabel("Summed Energy (keV)")
    points, vals = matrix_to_coords(label)
    plt.scatter(points[:,0],points[:,1], marker='s', c=vals, cmap='viridis', label="High Value Pixels", s=10)
    cbar = plt.colorbar()
    plt.xlim(0,10000)
    plt.ylim(0,10000)


## Methodology

### Baselines

In [ ]:
# My baseline model will be taking the two highest energy pixels

# Note there is no need for a training set because we are not training a model

# Load the data:
single_test = np.load("/mnt/home/taftkyle/indiv_data/singles/raw_single_test.npy")
double_test = np.load("/mnt/home/taftkyle/indiv_data/doubles/raw_double_test.npy")

# Load the labels:
single_test_labels = np.load("/mnt/home/taftkyle/indiv_data/singles/single_test_label.npy")
double_test_labels = np.load("/mnt/home/taftkyle/indiv_data/doubles/double_test_label.npy")

# Transform the labels:
single_test_labels = np.array([transform_label(label) for label in single_test_labels])
double_test_labels = np.array([transform_label(label, single = False) for label in double_test_labels])


# Concatenate the data:
test = np.concatenate((single_test, double_test))
test_labels = np.concatenate((single_test_labels, double_test_labels))

# Reuse the function from before to find the two highest energy pixels:
test_pred = np.zeros((test.shape[0], 4))
for i in range(test.shape[0]):
    test_pred[i] = transform_label(test[i], single = False)

# Evaluate the model using MSE and MAE:
from sklearn.metrics import mean_squared_error
print("Test RMSE: ", mean_squared_error(test_labels, test_pred)**0.5)
print("Test MAE: ", np.mean(np.abs(test_labels - test_pred)))

### Neural Network

### Convolutional Neural Network

In [ ]:
# ###### TRAINING CODE ######
# # Load modules
# print("Loading modules...")

# import tensorflow as tf
# from tensorflow import keras
# import numpy as np
# from tensorflow.keras.callbacks import ModelCheckpoint
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# # Prepare the data
# input_shape = (512, 512, 1)

# print("Reading in data...")
# # Read in data
# x_train_singles = np.load("/mnt/home/taftkyle/indiv_data/singles/raw_single_train.npy")
# x_train_double = np.load("/mnt/home/taftkyle/indiv_data/doubles/raw_double_train.npy")

# # Combine the data
# x_train = np.concatenate((x_train_singles, x_train_double))
# y_train = np.load("y_train.npy")

# print("x_train shape:", x_train.shape)
# print("y_train shape:", y_train.shape)

# #Shuffle data
# indices = np.arange(x_train.shape[0])
# np.random.shuffle(indices)
# x_train = x_train[indices]
# y_train = y_train[indices]

# # Define the model
# model = Sequential()
# model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Dropout(0.2))
# model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Dropout(0.4))
# model.add(Flatten())
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(4, activation='linear'))  # (x,y) coordinates

# # Compile the model
# model.compile(
#     optimizer=keras.optimizers.Adam(1e-3),
#     loss=keras.losses.MeanSquaredError(),
#     metrics=[
#         keras.metrics.MeanSquaredError(name="mse"),
#         keras.metrics.MeanAbsoluteError(name="mae"),
#     ],
# )

# #Train model
# print("Training model...")

# checkpoint_filepath = "cnn_best_model_CMSE492"
# checkpoint_callback = keras.callbacks.ModelCheckpoint(
#     checkpoint_filepath,
#     monitor="val_mae",
#     mode = 'min',
#     save_best_only=True,
# )

# history = model.fit(
#         x=x_train,
#         y=y_train,
#         batch_size=32,
#         epochs=100,
#         validation_split=0.1,
#         callbacks=[checkpoint_callback],
# )


# print("Model trained.")

# #Save model
# model.save('cnn_model_CMSE492')

# #Save history
# np.save("cnn_history_CMSE492.npy", history.history)

In [2]:
###### EVALUATION CODE ######
import numpy as np
np.load("cnn_history_CMSE492.npy", allow_pickle=True).item()

{'loss': [3018419.5, 1689657.75],
 'mse': [3018419.5, 1689657.75],
 'mae': [1216.6300048828125, 919.8373413085938],
 'val_loss': [852182.25, 1106993.75],
 'val_mse': [852182.25, 1106993.75],
 'val_mae': [700.7245483398438, 781.4419555664062]}

### Visual Transformer

In [ ]:
# ###### TRAINING CODE ######
# # Load modules
# print("Loading modules...")

# import numpy as np
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
# import tensorflow_addons as tfa

# # Prepare the data

# input_shape = (512, 512, 1)

# print("Reading in data...")
# # Read in data
# x_train_singles = np.load("/mnt/home/taftkyle/indiv_data/singles/raw_single_train.npy")
# x_train_double = np.load("/mnt/home/taftkyle/indiv_data/doubles/raw_double_train.npy")

# #Combine the data
# x_train = np.concatenate((x_train_singles, x_train_double))
# y_train = np.load("y_train.npy")

# print("x_train shape:", x_train.shape)
# print("y_train shape:", y_train.shape)

# #Shuffle data
# indices = np.arange(x_train.shape[0])
# np.random.shuffle(indices)
# x_train = x_train[indices]
# y_train = y_train[indices]

# learning_rate = 0.001
# weight_decay = 0.0001
# batch_size = 128
# num_epochs = 100
# image_size = 512  # We'll resize input images to this size
# patch_size = 32  # Size of the patches to be extract from the input images
# num_patches = (image_size // patch_size) ** 2
# projection_dim = 64
# num_heads = 4
# transformer_units = [
#     projection_dim * 2,
#     projection_dim,
# ]  # Size of the transformer layers
# transformer_layers = 8
# mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier


# # Use data augmentation
# data_augmentation = keras.Sequential(
#     [
#         layers.Normalization(),
#         layers.Resizing(image_size, image_size),
#     ],
#     name="data_augmentation",
# )
# # Compute the mean and the variance of the training data for normalization.
# data_augmentation.layers[0].adapt(x_train)


# # Implement multilayer perceptron (MLP)
# def mlp(x, hidden_units, dropout_rate):
#     for units in hidden_units:
#         x = layers.Dense(units, activation=tf.nn.gelu)(x)
#         x = layers.Dropout(dropout_rate)(x)
#     return x


# # Implement patch creation as a layer
# class Patches(layers.Layer):
#     def __init__(self, patch_size):
#         super().__init__()
#         self.patch_size = patch_size

#     def call(self, images):
#         batch_size = tf.shape(images)[0]
#         patches = tf.image.extract_patches(
#             images=images,
#             sizes=[1, self.patch_size, self.patch_size, 1],
#             strides=[1, self.patch_size, self.patch_size, 1],
#             rates=[1, 1, 1, 1],
#             padding="VALID",
#         )
#         patch_dims = patches.shape[-1]
#         patches = tf.reshape(patches, [batch_size, -1, patch_dims])
#         return patches


# # Display patches for a sample image
# import matplotlib.pyplot as plt

# plt.figure(figsize=(4, 4))
# image = x_train[np.random.choice(range(x_train.shape[0]))]
# plt.imshow(image.astype("uint8"))
# plt.axis("off")

# resized_image = tf.image.resize(
#     tf.convert_to_tensor([image]), size=(image_size, image_size)
# )
# print("Patch size:", patch_size)
# print("Image size:", image_size)
# print("Patches per image:", num_patches)
# print("Resized", resized_image.shape)
# patches = Patches(patch_size)(resized_image)
# print(f"Image size: {image_size} X {image_size}")
# print(f"Patch size: {patch_size} X {patch_size}")
# print(f"Patches per image: {patches.shape[1]}")
# print(f"Elements per patch: {patches.shape[-1]}")



# class PatchEncoder(layers.Layer):
#     def __init__(self, num_patches, projection_dim):
#         super().__init__()
#         self.num_patches = num_patches
#         self.projection = layers.Dense(units=projection_dim)
#         self.position_embedding = layers.Embedding(
#             input_dim=num_patches, output_dim=projection_dim
#         )

#     def call(self, patch):
#         positions = tf.range(start=0, limit=self.num_patches, delta=1)
#         encoded = self.projection(patch) + self.position_embedding(positions)
#         return encoded



# def create_vit_classifier():
#     print(input_shape)
#     inputs = layers.Input(shape=input_shape)
#     # Augment data.
#     augmented = data_augmentation(inputs)
#     # Create patches.
#     patches = Patches(patch_size)(augmented)
#     # Encode patches.
#     encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

#     # Create multiple layers of the Transformer block.
#     for _ in range(transformer_layers):
#         # Layer normalization 1.
#         x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
#         # Create a multi-head attention layer.
#         attention_output = layers.MultiHeadAttention(
#             num_heads=num_heads, key_dim=projection_dim, dropout=0.1
#         )(x1, x1)
#         # Skip connection 1.
#         x2 = layers.Add()([attention_output, encoded_patches])
#         # Layer normalization 2.
#         x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
#         # MLP.
#         x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
#         # Skip connection 2.
#         encoded_patches = layers.Add()([x3, x2])

#     # Create a [batch_size, projection_dim] tensor.
#     representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
#     representation = layers.Flatten()(representation)
#     representation = layers.Dropout(0.5)(representation)

#     # Add MLP.
#     features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)

#     # Output layers for peak localization and intensity prediction
#     peak_localization = layers.Dense(4, activation='linear')(features)  # (x, y) coordinates of peak centers

#     # Create the custom ViT model
#     model = keras.Model(inputs=inputs, outputs=peak_localization)

#     return model

# # Compile, train, and evaluate the mode

# def run_experiment(model):
#     optimizer = tfa.optimizers.AdamW(
#         learning_rate=learning_rate, weight_decay=weight_decay
#     )

#     model.compile(
#         optimizer=optimizer,
#         loss=keras.losses.MeanSquaredError(),
#         metrics=[
#             keras.metrics.MeanSquaredError(name="mse"),
#             keras.metrics.MeanAbsoluteError(name="mae"),
#         ],
#     )

#     checkpoint_filepath = "tmp/checkpoint/vit_checkpoint"
#     checkpoint_callback = keras.callbacks.ModelCheckpoint(
#         checkpoint_filepath,
#         monitor="val_mae",
#         save_best_only=True,
#         save_weights_only=True,
#     )

#     history = model.fit(
#         x=x_train,
#         y=y_train,
#         batch_size=batch_size,
#         epochs=num_epochs,
#         validation_split=0.1,
#         callbacks=[checkpoint_callback],
#     )

#     model.save("vit_model_CMSE492")
#     return history

# vit_classifier = create_vit_classifier()
# history = run_experiment(vit_classifier)

# np.save("vit_history_CMSE492.npy", history.history)

In [1]:
###### EVALUATION CODE ######
import numpy as np
np.load("history.npy", allow_pickle=True).item()

{'loss': [9161.853515625, 1063.215576171875],
 'mse': [9161.853515625, 1063.215576171875],
 'mae': [64.81576538085938, 24.205732345581055],
 'val_loss': [663.7801513671875, 388.2276916503906],
 'val_mse': [663.7801513671875, 388.2276916503906],
 'val_mae': [17.322500228881836, 14.838343620300293]}

## Results

_(How do your models compare? What did you find when you carried out your methods? Some of your code related to
presenting results/figures/data may be replicated from the methods section or may only be present in
this section. All of the plots that you plan on using for your presentation should be present in this
section)_

## Discussion and Conclusion

_(What did you learn from your results? What obstacles did you run into? What would you do differently next time? Clearly provide quantitative answers to your question(s)?  At least one of your questions should be answered with numbers.  That is, it is not sufficient to answer "yes" or "no", but rather to say something quantitative such as variable 1 increased roughly 10% for every 1 year increase in variable 2.)_

### References

_(List the source(s) for any data and/or literature cited in your project.  Ideally, this should be formatted using a formal citation format (MLA or APA or other, your choice!).   Multiple free online citation generators are available such as <a href="http://www.easybib.com/style">http://www.easybib.com/style</a>. **Important:** if you use **any** code that you find on the internet for your project you **must** cite it or you risk losing most/all of the points for you project.)_


Cade's code

ViT code

Image at top